In [3]:
from base import generator_gellmann
from base import generator_haar

def init_experiment(n):
    d = 2**n

    # Generate 6^n density matrices
    rho_list = generator_haar.generate_n_qubits_rho_haar(n)
    print(f"Generated {len(rho_list)} of {rho_list[0].shape} rho.")

    # Generate epsilon
    epsilon = generator_haar.random_unitary(d)
    print(f"Generated {epsilon.shape} epsilon.")

    # Generate K list
    unitary = generator_haar.haar(d)
    kraus_operators = generator_haar.generate_kraus_operators(unitary)
    print(f"Generated {len(kraus_operators)} of {kraus_operators[0].shape} kraus operators.")
    return rho_list, epsilon, kraus_operators

In [4]:
from base import epsilon_rho
def calculate_rho2_lists(rho_list, epsilon, kraus_operators):
    rho2 = []
    rho2_kraus = []
    for rho in rho_list:
        rho2.append(epsilon_rho.calculate_from_unitary(rho, epsilon))
        rho2_kraus.append(epsilon_rho.calculate_from_kraus_operators(rho, kraus_operators))
    return rho2, rho2_kraus

In [5]:
import numpy as np
import tensorflow as tf
def write_to_file(filename, data):
    """Write TensorFlow tensor data to a text file without truncation."""
    tensor_data = data.numpy() if isinstance(data, tf.Tensor) else data

    # Open the file and write the tensor data
    with open(filename, 'w') as f:
        if isinstance(data, np.ndarray):
            np.savetxt(f, data, fmt="%.6f")
        elif isinstance(data, list):
            for item in data:
                f.write(f"{item}\n")
        else:
            f.write(str(data))

In [33]:
import os
from base import optimize_algorithm
from base import metrics
experiment_folder = 'results/experiment_new/haar_random'

for num_qubits in range(1, 2):
    if (experiment_folder == ''):
        break
    else:
        write_folder = os.path.join(experiment_folder, str(num_qubits) + "_qubits")
        if not os.path.exists(write_folder):
            os.makedirs(write_folder)
    print(f"N={num_qubits}")

    #-----Init experiment-----
    rho_list, epsilon, kraus_operators = init_experiment(num_qubits)

    #-----Learn kraus operators-----
    kraus_operators_res, cost_dict = optimize_algorithm.optimize_adam_kraus_set(rho_list, epsilon, kraus_operators, num_qubits, 0.05, num_loop=200)
    
    #-----Calculate result data-----
    rho3_list = epsilon_rho.calculate_set_from_kraus_operators(kraus_operators_res, rho_list, epsilon)
    rho2_list, rho2_kraus_list = calculate_rho2_lists(rho_list, epsilon, kraus_operators_res)
    
    mean_fidelity_rho_rho3 = metrics.mean_fidelity(rho3_list, rho_list)
    mean_fidelity_rho2_rho2 = metrics.mean_fidelity(rho2_kraus_list, rho2_list)

    #-----Write to folder-----
    write_to_file(os.path.join(write_folder, "rho_list.txt"), rho_list)
    write_to_file(os.path.join(write_folder,"epsilon.txt"), epsilon)
    
    write_to_file(os.path.join(write_folder,"kraus_operators.txt"), kraus_operators_res)
    write_to_file(os.path.join(write_folder,"cost_dict.txt"), cost_dict)

    write_to_file(os.path.join(write_folder,"rho3_list.txt"), rho3_list)
    write_to_file(os.path.join(write_folder,"rho2_list.txt"), rho2_list)
    write_to_file(os.path.join(write_folder,"rho2_kraus_list.txt"), rho2_kraus_list)

    write_to_file(os.path.join(write_folder,"mean_fidelity_rho_rho3.txt"), mean_fidelity_rho_rho3.numpy())
    write_to_file(os.path.join(write_folder,"mean_fidelity_rho2_rho2.txt"), mean_fidelity_rho2_rho2.numpy())

    
    

N=1
Generated 6 of (2, 2) rho.
Generated (2, 2) epsilon.
Generated 2 of (2, 2) kraus operators.
tf.Tensor(0.7278318369400308, shape=(), dtype=float64)
tf.Tensor(0.6304117608825711, shape=(), dtype=float64)
tf.Tensor(0.5348768064810686, shape=(), dtype=float64)
tf.Tensor(0.44560711775143513, shape=(), dtype=float64)
tf.Tensor(0.36550983198839204, shape=(), dtype=float64)
tf.Tensor(0.29586677087729224, shape=(), dtype=float64)
tf.Tensor(0.23661883745100756, shape=(), dtype=float64)
tf.Tensor(0.1869265844181365, shape=(), dtype=float64)
tf.Tensor(0.14571726461830814, shape=(), dtype=float64)
tf.Tensor(0.11199601491654831, shape=(), dtype=float64)
tf.Tensor(0.08490959058287477, shape=(), dtype=float64)
tf.Tensor(0.06369012894855294, shape=(), dtype=float64)
tf.Tensor(0.047590379380766705, shape=(), dtype=float64)
tf.Tensor(0.03585252813578369, shape=(), dtype=float64)
tf.Tensor(0.027710278075259383, shape=(), dtype=float64)
tf.Tensor(0.02241187179056581, shape=(), dtype=float64)
tf.Tensor(

In [61]:
experiment_folder = 'results/experiment_new/haar_random_matrix_comparison'
rho_test = generator_haar.generate_rho_haar(1)

rho2 = epsilon_rho.calculate_from_unitary(rho_test, epsilon)

rho2_kraus = epsilon_rho.calculate_from_kraus_operators(rho_test, kraus_operators_res)

rho3 = epsilon_rho.calculate_from_unitary_dagger(rho2_kraus, epsilon)

print(rho_test)
print(rho2)
if (experiment_folder!=''):
    write_to_file(os.path.join(experiment_folder,"rho.txt"), rho_test)
    write_to_file(os.path.join(experiment_folder,"rho3.txt"), rho3.numpy())
    write_to_file(os.path.join(experiment_folder,"rho2.txt"), rho2)
    write_to_file(os.path.join(experiment_folder,"rho2_kraus.txt"), rho2_kraus.numpy())
    write_to_file(os.path.join(experiment_folder,"fide03.txt"), metrics.compilation_trace_fidelity(rho_test, rho3))
    write_to_file(os.path.join(experiment_folder,"fide22.txt"), metrics.compilation_trace_fidelity(rho2, rho2_kraus))
    

[[ 0.93912925+6.30306403e-18j -0.22902975-6.86358331e-02j]
 [-0.22902975+6.86358331e-02j  0.06087075+4.26053067e-18j]]
[[0.21482016+2.13941013e-17j 0.4049217 +6.86358331e-02j]
 [0.4049217 -6.86358331e-02j 0.78517984-1.56381019e-17j]]
